# Input pipeline into Keras

In this notebook, we will look at how to read large datasets, datasets that may not fit into memory, using TensorFlow. We can use the tf.data pipeline to feed data to Keras models that use a TensorFlow backend.

Let's start off with the Python imports that we need.

In [4]:
import os, json, math
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.0.0-alpha0


## Locating the CSV files

We will start with the CSV files that we wrote out in the [first notebook](../01_explore/taxifare.iypnb) of this sequence. Just so you don't have to run the notebook, we saved a copy in ../data

In [3]:
!ls -l ../data/*.csv

-rw-r--r-- 1 jupyter jupyter  85978 Jun  1 18:03 ../data/taxi-test.csv
-rw-r--r-- 1 jupyter jupyter 402560 Jun  1 18:03 ../data/taxi-train.csv
-rw-r--r-- 1 jupyter jupyter  85797 Jun  1 18:03 ../data/taxi-valid.csv


## Use tf.data to read the CSV files

See the documentation for [make_csv_dataset](https://www.tensorflow.org/api_docs/python/tf/data/experimental/make_csv_dataset).
If you have TFRecords (which is recommended), use [make_batched_features_dataset](https://www.tensorflow.org/api_docs/python/tf/data/experimental/make_batched_features_dataset) instead.

In [5]:
CSV_COLUMNS  = ['fare_amount', 
                'pickup_longitude', 'pickup_latitude', 
                'dropoff_longitude', 'dropoff_latitude', 
                'passenger_count', 'key']
LABEL_COLUMN = 'fare_amount'
DEFAULTS     = [[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],['na']]

In [12]:
# load the training data
def load_dataset(pattern):
  return tf.data.experimental.make_csv_dataset(pattern, 1, CSV_COLUMNS, DEFAULTS)

tempds = load_dataset('../data/taxi-train*')
print(tempds)

<PrefetchDataset shapes: OrderedDict([(fare_amount, (1,)), (pickup_longitude, (1,)), (pickup_latitude, (1,)), (dropoff_longitude, (1,)), (dropoff_latitude, (1,)), (passenger_count, (1,)), (key, (1,))]), types: OrderedDict([(fare_amount, tf.float32), (pickup_longitude, tf.float32), (pickup_latitude, tf.float32), (dropoff_longitude, tf.float32), (dropoff_latitude, tf.float32), (passenger_count, tf.float32), (key, tf.string)])>


Note that this is a prefetched dataset. If you loop over the dataset, you'll get the rows one-by-one. Let's convert each row into a Python dictionary:

In [11]:
# print a few of the rows
for n, data in enumerate(tempds):
    row_data = {k: v.numpy() for k,v in data.items()}
    print(n, row_data)
    if n > 2:
        break

0 {'dropoff_longitude': array([-73.96857], dtype=float32), 'passenger_count': array([2.], dtype=float32), 'fare_amount': array([6.], dtype=float32), 'pickup_longitude': array([-73.97851], dtype=float32), 'dropoff_latitude': array([40.760876], dtype=float32), 'pickup_latitude': array([40.75231], dtype=float32), 'key': array([b'1388'], dtype=object)}
1 {'dropoff_longitude': array([-74.01276], dtype=float32), 'passenger_count': array([1.], dtype=float32), 'fare_amount': array([5.], dtype=float32), 'pickup_longitude': array([-74.01003], dtype=float32), 'dropoff_latitude': array([40.707954], dtype=float32), 'pickup_latitude': array([40.705784], dtype=float32), 'key': array([b'2525'], dtype=object)}
2 {'dropoff_longitude': array([-73.95558], dtype=float32), 'passenger_count': array([5.], dtype=float32), 'fare_amount': array([4.5], dtype=float32), 'pickup_longitude': array([-73.945946], dtype=float32), 'dropoff_latitude': array([40.779686], dtype=float32), 'pickup_latitude': array([40.77777],

What we really need is a dictionary of features + a label. So, we have to do two things to the above dictionary. (1) remove the unwanted column "key" and (2) keep the label separate from the features.

In [13]:
# get features, label
def features_and_labels(row_data):
    for unwanted_col in ['key']:
        row_data.pop(unwanted_col)
    label = row_data.pop(LABEL_COLUMN)
    return row_data, label  # features, label

# print a few rows to make it sure works
for n, data in enumerate(tempds):
    row_data = {k: v.numpy() for k,v in data.items()}
    features, label = features_and_labels(row_data)
    print(n, label, features)
    if n > 2:
        break

0 [7.3] {'dropoff_longitude': array([-73.97203], dtype=float32), 'passenger_count': array([1.], dtype=float32), 'pickup_longitude': array([-73.977], dtype=float32), 'dropoff_latitude': array([40.764313], dtype=float32), 'pickup_latitude': array([40.74728], dtype=float32)}
1 [5.] {'dropoff_longitude': array([-73.97982], dtype=float32), 'passenger_count': array([1.], dtype=float32), 'pickup_longitude': array([-73.98815], dtype=float32), 'dropoff_latitude': array([40.751232], dtype=float32), 'pickup_latitude': array([40.74935], dtype=float32)}
2 [29.5] {'dropoff_longitude': array([-73.960144], dtype=float32), 'passenger_count': array([1.], dtype=float32), 'pickup_longitude': array([-73.86369], dtype=float32), 'dropoff_latitude': array([40.687412], dtype=float32), 'pickup_latitude': array([40.769897], dtype=float32)}
3 [7.] {'dropoff_longitude': array([-73.970825], dtype=float32), 'passenger_count': array([5.], dtype=float32), 'pickup_longitude': array([-73.96219], dtype=float32), 'dropoff

## Batching

Let's do both (loading, features_label)
in our load_dataset function, and also add batching.

In [17]:
def load_dataset(pattern, batch_size):
  return (
      tf.data.experimental.make_csv_dataset(pattern, batch_size, CSV_COLUMNS, DEFAULTS)
             .map(features_and_labels) # features, label
  )

# try changing the batch size and watch what happens.
tempds = load_dataset('../data/taxi-train*', batch_size=2)
print(list(tempds.take(3))) # truncate and print as a list 

[(OrderedDict([('pickup_longitude', <tf.Tensor: id=1015, shape=(2,), dtype=float32, numpy=array([-73.974304, -73.97505 ], dtype=float32)>), ('pickup_latitude', <tf.Tensor: id=1014, shape=(2,), dtype=float32, numpy=array([40.75003, 40.73255], dtype=float32)>), ('dropoff_longitude', <tf.Tensor: id=1012, shape=(2,), dtype=float32, numpy=array([-73.87089 , -73.981155], dtype=float32)>), ('dropoff_latitude', <tf.Tensor: id=1011, shape=(2,), dtype=float32, numpy=array([40.7719  , 40.741596], dtype=float32)>), ('passenger_count', <tf.Tensor: id=1013, shape=(2,), dtype=float32, numpy=array([5., 5.], dtype=float32)>)]), <tf.Tensor: id=1016, shape=(2,), dtype=float32, numpy=array([28.9,  4.9], dtype=float32)>), (OrderedDict([('pickup_longitude', <tf.Tensor: id=1021, shape=(2,), dtype=float32, numpy=array([-73.96097, -73.98101], dtype=float32)>), ('pickup_latitude', <tf.Tensor: id=1020, shape=(2,), dtype=float32, numpy=array([40.772446, 40.741745], dtype=float32)>), ('dropoff_longitude', <tf.Tens

## Shuffling

When training a deep learning model in batches over multiple workers, it is helpful if we shuffle the data. That way, different workers will be working on different parts of the input file at the same time, and so averaging gradients across workers will help. Also, during training, we will need to read the data indefinitely.

In [23]:
def load_dataset(pattern, batch_size=1, mode=tf.estimator.ModeKeys.EVAL):
  dataset = (tf.data.experimental.make_csv_dataset(pattern, batch_size, CSV_COLUMNS, DEFAULTS)
             .map(features_and_labels) # features, label
             .cache())
  if mode == tf.estimator.ModeKeys.TRAIN:
        dataset = dataset.shuffle(1000).repeat()
  dataset = dataset.prefetch(1) # take advantage of multi-threading; 1=AUTOTUNE
  return dataset

tempds = load_dataset('../data/taxi-train*', 2, tf.estimator.ModeKeys.TRAIN)
print(list(tempds.take(1)))
tempds = load_dataset('../data/taxi-valid*', 2, tf.estimator.ModeKeys.EVAL)
print(list(tempds.take(1)))

[(OrderedDict([('pickup_longitude', <tf.Tensor: id=2201, shape=(2,), dtype=float32, numpy=array([-73.97941, -74.00601], dtype=float32)>), ('pickup_latitude', <tf.Tensor: id=2200, shape=(2,), dtype=float32, numpy=array([40.781647, 40.73576 ], dtype=float32)>), ('dropoff_longitude', <tf.Tensor: id=2198, shape=(2,), dtype=float32, numpy=array([-73.95575 , -73.986176], dtype=float32)>), ('dropoff_latitude', <tf.Tensor: id=2197, shape=(2,), dtype=float32, numpy=array([40.77253, 40.73058], dtype=float32)>), ('passenger_count', <tf.Tensor: id=2199, shape=(2,), dtype=float32, numpy=array([1., 1.], dtype=float32)>)]), <tf.Tensor: id=2202, shape=(2,), dtype=float32, numpy=array([7.3, 6.9], dtype=float32)>)]
[(OrderedDict([('pickup_longitude', <tf.Tensor: id=2299, shape=(2,), dtype=float32, numpy=array([-73.975105, -73.95398 ], dtype=float32)>), ('pickup_latitude', <tf.Tensor: id=2298, shape=(2,), dtype=float32, numpy=array([40.736317, 40.770515], dtype=float32)>), ('dropoff_longitude', <tf.Tenso

In the next notebook, we will build the model using this input pipeline.

Copyright 2019 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.